In [19]:
import numpy as np
from Bio import AlignIO
from Bio.Seq import Seq
from io import StringIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import MafftCommandline
import numpy as np
import pandas as pd
import collections as co

In [20]:
infile = '../../B.csv'   
upload = pd.read_csv(infile, sep = ',', na_filter = False, header = None)
upload.columns = ['accession', 'strain', 'segment', 'protein', 'genus', 'subtype', 'date', 'host', 'curation', 'genome']
upload.query('curation == "Pass"', inplace = True)
upload.set_index('accession', inplace = True)

In [22]:
cluster = pd.read_csv('Output/cluster.csv', sep = ',', na_filter = False, index_col = 0, header = 0)
num = cluster['cluster'].max()
z = co.defaultdict(list)
x = co.defaultdict(str)
z[0] = co.defaultdict(list)
for j in range(num):

    query = cluster.query('cluster == @j')
    match = query.index.values.tolist()
    sub = upload.filter(items = match, axis=0)

    fasta = sub[['genome']].copy()
    fasta.to_csv('genome.fasta', header=None, index=True, sep='\n', mode='w')

    mafft_cline = MafftCommandline(input='genome.fasta', thread=8)
    stdout, stderr = mafft_cline()
    #align = AlignIO.read(StringIO(stdout), "fasta")
    #count = AlignIO.write(align, "result.faa", "fasta")

    
    for i in stdout.split('\n'):
        if i == '':
            pass
        elif i[0] == '>':
            accession = i
        else:
            x[accession] += i

    y = pd.DataFrame.from_dict(x, orient='index', columns=['alignment'])
    y.index.rename('accession', inplace=True)

    x.clear()
    
    z[0][j] = y
    
#result = pd.concat(z)
#result.index.set_names(["cluster", "accession"], inplace=True)
#result.reset_index(level = "cluster", inplace=True)  
#result.sort_values(by=['cluster', 'accession', 'alignment'], inplace = True)

KeyboardInterrupt: 